In [1]:
import os
import sys
sys.path.append('D:\实习\研报复现\QuantDev\ymtool')
import numpy as np
import pandas as pd
from tqdm import tqdm
import statsmodels.api as sm

import Alpha as alpha

# GTR

In [3]:
turnover =  alpha.load_data('pv\\stock_turnover').replace(0,np.nan)

GTR = alpha.mvind_neutral(turnover.pct_change().rolling(20).std().dropna(axis=0,how='all'))

C:\Users\Devin\AppData\Local\Temp\ipykernel_2652\1132332965.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  GTR = alpha.mvind_neutral(turnover.pct_change().rolling(20).std().dropna(axis=0,how='all'))
100%|██████████████████████████████████████████████████████████████████████████████| 3683/3683 [00:52<00:00, 69.52it/s]


In [4]:
alpha.to_fctDataBase(GTR,'GTR')
fct = alpha.load_fct('GTR')
alpha.Alpha(fct, 'GTR', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\GTR_W.xlsx


# 加强作用

In [5]:
def nonneg(fct):
    min_val = fct.min(axis=1)
    nonneg_fct = fct.sub(min_val,axis=0)
    return nonneg_fct

Turn20 = alpha.mvind_neutral(turnover.rolling(20).mean().dropna(axis=0,how='all'))
STR = alpha.mvind_neutral(turnover.rolling(20).std().dropna(axis=0,how='all'))
PctTurn20 = alpha.mvind_neutral((turnover.div(turnover.rolling(40).mean().shift(20),axis=0)-1).rolling(20).mean().dropna(axis=0,how='all'))
SCR = alpha.mvind_neutral((turnover.rolling(20).std().div(turnover.rolling(40).std().shift(20),axis=0)-1).dropna(axis=0,how='all'))


Turn20_Turbo = nonneg(alpha.zscore(Turn20)).rank(axis=1,pct=True)+(nonneg(alpha.zscore(GTR))).rank(axis=1,pct=True)
STR_Turbo = nonneg(alpha.zscore(STR)).rank(axis=1,pct=True)+(nonneg(alpha.zscore(GTR))).rank(axis=1,pct=True)
PctTurn20_Turbo = nonneg(alpha.zscore(PctTurn20)).rank(axis=1,pct=True)+(nonneg(alpha.zscore(GTR))).rank(axis=1,pct=True)
SCR_Turbo = nonneg(alpha.zscore(SCR)).rank(axis=1,pct=True)+(nonneg(alpha.zscore(GTR))).rank(axis=1,pct=True)

100%|██████████████████████████████████████████████████████████████████████████████| 3644/3644 [00:45<00:00, 80.60it/s]


In [6]:
Turn20_TurboNeu = alpha.mvind_neutral(Turn20_Turbo.dropna(axis=0,how='all'))
STR_TurboNeu = alpha.mvind_neutral(STR_Turbo.dropna(axis=0,how='all'))
PctTurn20_TurboNeu = alpha.mvind_neutral(PctTurn20_Turbo.dropna(axis=0,how='all'))
SCR_TurboNeu = alpha.mvind_neutral(SCR_Turbo.dropna(axis=0,how='all'))

100%|██████████████████████████████████████████████████████████████████████████████| 3644/3644 [00:51<00:00, 70.17it/s]


In [7]:
alpha.to_fctDataBase(Turn20_TurboNeu,'Turn20_TurboNeu')
fct = alpha.load_fct('Turn20_TurboNeu')
alpha.Alpha(fct, 'Turn20_TurboNeu', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\Turn20_TurboNeu_W.xlsx


In [8]:
alpha.to_fctDataBase(STR_Turbo,'STR_TurboNeu')
fct = alpha.load_fct('STR_TurboNeu')
alpha.Alpha(fct, 'STR_TurboNeu', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\STR_TurboNeu_W.xlsx


In [9]:
alpha.to_fctDataBase(PctTurn20_Turbo,'PctTurn20_TurboNeu')
fct = alpha.load_fct('PctTurn20_TurboNeu')
alpha.Alpha(fct, 'PctTurn20_TurboNeu', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\PctTurn20_TurboNeu_W.xlsx


In [10]:
alpha.to_fctDataBase(SCR_Turbo,'SCR_TurboNeu')
fct = alpha.load_fct('SCR_TurboNeu')
alpha.Alpha(fct, 'SCR_TurboNeu', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\SCR_TurboNeu_W.xlsx


# 纯净相加

In [11]:
tnvpctchange = turnover.sub(turnover.shift(1),axis=0).div(turnover.shift(1)+1e-6,axis=0).dropna(axis=0,how='all')

In [12]:
res = []
for idx in tqdm(tnvpctchange.index):
    temp = pd.concat([turnover.loc[idx],tnvpctchange.loc[idx]],axis=1).dropna()

    X = temp.iloc[:,1:]
    X = sm.add_constant(X)
    y = temp.iloc[:,0]

    # 创建模型
    model = sm.OLS(y, X)

    # 拟合模型
    results = model.fit()

    res.append(results.resid)

pureTnv = pd.concat(res,axis=1).T
pureTnv.index = tnvpctchange.index

100%|█████████████████████████████████████████████████████████████████████████████| 3702/3702 [00:10<00:00, 357.13it/s]


In [14]:
res = []
for idx in tqdm(tnvpctchange.index):
    temp = pd.concat([tnvpctchange.loc[idx],turnover.loc[idx]],axis=1).dropna()

    X = temp.iloc[:,1:]
    X = sm.add_constant(X)
    y = temp.iloc[:,0]

    # 创建模型
    model = sm.OLS(y, X)

    # 拟合模型
    results = model.fit()

    res.append(results.resid)

pureTnvpct = pd.concat(res,axis=1).T
pureTnvpct.index = tnvpctchange.index

100%|█████████████████████████████████████████████████████████████████████████████| 3702/3702 [00:10<00:00, 355.18it/s]


In [15]:
TPS_Turbo = alpha.zscore(alpha.mvind_neutral(pureTnv.rolling(20).mean().dropna(axis=0,how='all')))+\
alpha.zscore(alpha.mvind_neutral(pureTnvpct.rolling(20).std().dropna(axis=0,how='all')))

100%|██████████████████████████████████████████████████████████████████████████████| 3683/3683 [01:15<00:00, 48.91it/s]


In [16]:
SPS_Turbo = alpha.zscore(alpha.mvind_neutral(pureTnv.rolling(20).std().dropna(axis=0,how='all')))+\
alpha.zscore(alpha.mvind_neutral(pureTnvpct.rolling(20).std().dropna(axis=0,how='all')))

100%|██████████████████████████████████████████████████████████████████████████████| 3683/3683 [01:16<00:00, 48.40it/s]


In [17]:
alpha.to_fctDataBase(TPS_Turbo,'TPS_Turbo')
fct = alpha.load_fct('TPS_Turbo')
alpha.Alpha(fct, 'TPS_Turbo', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\TPS_Turbo_W.xlsx


In [19]:
alpha.to_fctDataBase(SPS_Turbo,'SPS_Turbo')
fct = alpha.load_fct('SPS_Turbo')
alpha.Alpha(fct, 'SPS_Turbo', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\SPS_Turbo_W.xlsx
